# The deAlmeida Overland Flow Component 

This notebook illustrates running the deAlmeida overland flow component in an extremely simple-minded way on a real topography, then shows it creating a flood sequence along an inclined surface with an oscillating water surface at one end.

First, import what we'll need:

In [ ]:
from landlab.components.overland_flow import OverlandFlow
from landlab.io import esri_ascii

Pick the initial and run conditions

In [ ]:
run_time = 100  # duration of run, (s)
h_init = 0.1  # initial thin layer of water (m)
n = 0.01  # roughness coefficient, (s/m^(1/3))
g = 9.8  # gravity (m/s^2)
alpha = 0.7  # time-step factor (nondimensional; from Bates et al., 2010)
u = 0.4  # constant velocity (m/s, de Almeida et al., 2012)
run_time_slices = (10, 50, 100)

Elapsed time starts at 1 second. This prevents errors when setting our boundary conditions.

In [ ]:
elapsed_time = 1.0

Use Landlab methods to import an ARC ascii grid, and load the data into the field that the component needs to look at to get the data. This loads the elevation data, z, into a "field" in the grid itself, defined on the nodes.

In [ ]:
with open("Square_TestBasin.asc") as fp:
    grid = esri_ascii.load(fp, name="topographic__elevation", at="node")
grid.set_closed_boundaries_at_grid_edges(True, True, True, True)

We can get at this data with this syntax:

Note that the boundary conditions for this grid mainly got handled with the final line of those three, but for the sake of completeness, we should probably manually "open" the outlet. We can find and set the outlet like this:

In [ ]:
# This DEM was generated using Landlab and the outlet node ID was known
my_outlet_node = 100
grid.status_at_node[my_outlet_node] = grid.BC_NODE_IS_FIXED_VALUE

Now initialize a couple more grid fields that the component is going to need:

In [ ]:
grid.add_zeros("surface_water__depth", at="node")  # water depth (m)

In [ ]:
grid.at_node["surface_water__depth"] += h_init

Let's look at our watershed topography

In [ ]:
grid.imshow("topographic__elevation")

Now instantiate the component itself

In [ ]:
# for stability in steeper environments, we set the steep_slopes flag to True
of = OverlandFlow(grid, steep_slopes=True)

Now we're going to run the loop that drives the component:

In [ ]:
of.run_one_step(dt=run_time)

In [ ]:
grid.imshow("surface_water__depth", cmap="Blues")

Now let's get clever, and run a set of time slices:

In [ ]:
dt = 10.0
for step in trange(11):
    time = step * dt

    of.run_one_step(dt=dt)

    if step in (1, 5, 10):
        plt.figure()
        grid.imshow("surface_water__depth", cmap="Blues")        plt.title(f"Time = {time} s")